# 🧠 Mastering LLM Fine-Tuning with TRL – Supervised Fine-Tuning

Welcome! This notebook is part of a tutorial series where you'll learn how to fine-tune Large Language Models (LLMs) using 🤗 TRL.
We introduce key concepts, set up the required tools, and use techniques like Supervised Fine-Tuning (SFT) and Group-Relative Policy Optimization (GRPO).

## 📋 Prerequisites

Before you begin, make sure you have the following:

* A working knowledge of Python and PyTorch
* A basic understanding of machine learning and deep learning concepts
* Access to a GPU accelerator – this notebook is designed to run with **at least 16GB of GPU memory**, such as what is available for free on [Google Colab](https://colab.research.google.com)
* The `trl` library installed – this tutorial has been tested with **TRL version 0.17**
  If you don’t have `trl` installed yet, you can install it by running the following code block:

In [ ]:
%pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━ 53.9/127.9 MB 229.4 MB/s eta 0:00:01

* A [Hugging Face account](https://huggingface.co) with a configured access token. If needed, run the following code.
This will prompt you to enter your Hugging Face access token. You can generate one from your Hugging Face account settings under [Access Tokens](https://huggingface.co/settings/tokens). The token must have `Write access to contents/settings of all repos under your personal namespace`

In [2]:
from huggingface_hub import notebook_login
notebook_login()

## 🔄 Quick Recap of the Last Session

In the previous session, we explored the foundational concepts behind training and fine-tuning Large Language Models (LLMs). Here's a brief summary:

* LLMs operate on sequences of integers known as *tokens*. Given a sequence, they predict the probability distribution of the next token in the sequence.
* The first phase of training an LLM is called **pretraining**. This involves training the model on a massive corpus of unlabeled text data.
* The output of pretraining is a **base model**, which has learned general language patterns but isn’t specialized for specific tasks.
* To adapt the base model for a particular use case—like building a chatbot—we need to **fine-tune** (or post-train) it on a dataset of conversations.
* Many high-quality conversational datasets are available publicly on the [Hugging Face Hub](https://huggingface.co/datasets).
* These datasets are often not in a format that's directly usable for training, so **data preprocessing** is usually required.

Now that we're on the same page, let's dive into the next session! First Let's load our base model and tokenizer.


We'll be using a different model from the first session: `SmolLM2-360M`.


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M", device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

As before, let's define a chat template and make the necessary modifications to the template and tokenizer.

In [4]:
tokenizer.chat_template = """{{- '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}
{%- for message in messages %}
    {{- '<|im_start|>' + message.role + '\n' + message.content + '<|im_end|>' + '\n' }}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|im_start|>assistant\n' }}
{%- endif %}"""
tokenizer.eos_token = "<|im_end|>"
model.config.eos_token_id = tokenizer.eos_token_id
model.generation_config.eos_token_id = tokenizer.eos_token_id

Remember, configuring a chat template doesn't make the model capable of chatting.
It only gives the ability to format inputs in a dialogue structure; the model still needs to be fine-tuned on conversational data to respond like a chatbot.

In [5]:
from transformers import pipeline

pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

question = "What does it mean for a matrix to be invertible?"
prompt = [{"role": "user", "content": question}]
pipeline(prompt, max_new_tokens=50)[0]["generated_text"]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'role': 'user',
  'content': 'What does it mean for a matrix to be invertible?'},
 {'role': 'assistant',
  'content': 'What does it mean for a matrix to be invertible?��boutiqueassistant\nWhat does it mean for a matrix to be invertible?��boutiqueassistant\nWhat does it mean for a matrix to be invertible?'}]

To make the exercise more engaging, we'll use a custom dataset I've created especially for you. Instead of the usual back-and-forth between a lazy user and a helpful assistant, I've spiced things up with dialogues between Rick and Morty. If this reference doesn't ring a bell - sorry for you!

So what we're going to do is basically train a Rick to respond to Morty.

![](https://cdn-uploads.huggingface.co/production/uploads/631ce4b244503b72277fc89f/m9fHggYpFjil8L55a3UNt.png)

In [6]:
from datasets import load_dataset

dataset = load_dataset("qgallouedec/rick-science", split="train")
dataset[0]

README.md:   0%|          | 0.00/351 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1183 [00:00<?, ? examples/s]

{'question': 'How does the velocity of a car change if it accelerates uniformly from rest with an acceleration of 2 m/s² for 5 seconds?',
 'reasoning': "Alright, let's dissect this snooze-fest of a question. Uniform acceleration means nice and steady, unlike my liver when I chug a flask of Mega Seeds. Starting from rest, so initial velocity is zero. Geez, way to state the obvious. I swear, if someone explains 'rest' one more time, I'm gonna lose my mind. So, we've got this precious little acceleration of 2 m/s² — what, did it get scared of the bigger numbers? Multiply that by the 5 seconds it manages to accelerate without falling apart, and we get the final velocity. It's a simple product of time and acceleration. Velocity = acceleration × time, blah, blah, blah. It's basic physics, but apparently, the geniuses of the Universe need me to hold their hand through it.",
 'answer': "Okay, Morty! Listen up, 'cause I'm only saying this once, alright? Your precious car goes from 'sitting-on-i

Similarly, you can see that the dataset needs to be pre-formatted. So, as before, we write and apply the function to format the dataset.

In [7]:
def to_conversation(example):
    return {
        "messages": [
            {"role": "user", "content": example["question"]},
            {"role": "assistant", "content": f"<think>{example['reasoning']}</think> {example['answer']}"},
        ]
    }

dataset = dataset.map(to_conversation, remove_columns=dataset.column_names)
dataset[0]

Map:   0%|          | 0/1183 [00:00<?, ? examples/s]

{'messages': [{'content': 'How does the velocity of a car change if it accelerates uniformly from rest with an acceleration of 2 m/s² for 5 seconds?',
   'role': 'user'},
  {'content': "<think>Alright, let's dissect this snooze-fest of a question. Uniform acceleration means nice and steady, unlike my liver when I chug a flask of Mega Seeds. Starting from rest, so initial velocity is zero. Geez, way to state the obvious. I swear, if someone explains 'rest' one more time, I'm gonna lose my mind. So, we've got this precious little acceleration of 2 m/s² — what, did it get scared of the bigger numbers? Multiply that by the 5 seconds it manages to accelerate without falling apart, and we get the final velocity. It's a simple product of time and acceleration. Velocity = acceleration × time, blah, blah, blah. It's basic physics, but apparently, the geniuses of the Universe need me to hold their hand through it.</think> Okay, Morty! Listen up, 'cause I'm only saying this once, alright? Your pr

At this point, we have a model ready for training and a dataset ready to go. This is where TRL comes in—it provides the `SFTTrainer`, a trainer designed to fine-tune the model using the dataset.

**But what is SFT?**


## 🕵️ Supervised Fine-Tuning (SFT)

SFT stands for **Supervised Fine-Tuning**. There’s nothing particularly revolutionary about the SFT method. It’s simply training the model to predict the next token in a supervised way. Just like in pretraining, we minimize the cross-entropy loss between the model’s predicted distribution and the actual next token. The key difference is that, in SFT, the model is trained on **curated**, labeled data — often conversational or instruction-following examples.

I'm going to break from the usual approach and show you the training process *before* diving into the explanation. Why?

1. You'll get an immediate feel for what the training actually does.
2. The training can run in the background while I walk you through what's happening.

Without explanation, here's the training code block.

In [1]:
from trl import SFTTrainer, SFTConfig

args = SFTConfig(
    output_dir="data/SmolLM2-360M-Rickified",
    gradient_checkpointing=True,
    bf16=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_length=650,
    logging_steps=10,
    run_name="SmolLM2-360M-Rickified",
)

trainer = SFTTrainer(
    model=model,
    args=args,
    processing_class=tokenizer,
    train_dataset=dataset,
)

ModuleNotFoundError: No module named 'trl'

In [ ]:
trainer.train()

Now that we've trained our model, it's time to share it with the world! To do so, simply push it to the Hugging Face Hub using `push_to_hub`.

In [ ]:
trainer.push_to_hub(dataset_name="qgallouedec/rick-science")

It's very important to remember that what training LLM, VRAM is the backbone of the battle. This is especially important in our case, as we’re working with limited compute resources. I designed this notebook to be runnable on the free version of Colab, which only provide a GPUs with 16GB of memory. As a result, we must manage GPU memory carefully to avoid running out. This constraint also presents a valuable opportunity to learn about GPU memory usage and optimization—skills that are essential when training large language models (LLMs).

### 🥛 What consumes GPU memory?

When you profile GPU memory usage during training, you get a chart like this one:

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/train_memory/colorized_training_profile.png)

As shown in the graph, several components occupy GPU memory:

* **Model**: The neural network itself, loaded into memory.
* **Optimizer**: This usually takes up twice as much memory as the model.
* **Activations**: These are the intermediate outputs from each layer during the forward pass.
* **Gradients**: The derivatives of the loss with respect to the activations, used during backpropagation.
* **Optimizer states**: Temporary variables needed by the optimizer during training.

### 🤏 Use a smaller model

You might have noticed that base models are often released in multiple sizes. Larger models tend to perform better, but they also consume significantly more memory. Choosing a smaller model is the most impactful optimization you can make—it reduces not just the model size, but also the memory used by the optimizer and activations.

For this notebook, we’ll be using `SmolLM2-360M`.

### 📐 Handle the sequence length carefully

The memory required for activations is directly proportional to the number of tokens in a batch. The number of tokens in a batch, in turn, depends on two factors: the *batch size* and the *sequence length*. Doubling the sequence length will double the memory needed for activations, and similarly, doubling the batch size will also double the memory required for activations.

![](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/why_you_should_truncate.png)

And remember, whether you hit an Out Of Memory (OOM) error only depends on the longest sequence in the entire dataset. If, at some point during training, you encounter a sequence longer than what the GPU can handle, you'll get an OOM error, and the training will need to be restarted from scratch. So, it's important to control this maximum sequence length, and truncate any sequences that exceed it.

To get a better sense of what could be a reasonable value for the maximum sequence length, you can check the distribution of sequence lengths in your dataset. You can do this by running the following code:

In [ ]:
import matplotlib.pyplot as plt

input_ids = [tokenizer.apply_chat_template(example["messages"]) for example in dataset]
lens = [len(ids) for ids in input_ids]

plt.hist(lens, bins=50)
plt.xlabel("Length of input_ids")
plt.ylabel("Number of examples")
plt.title("Distribution of input_ids length")
plt.show()

### Handle the batch size carefully

As explained earlier, batch size directly impacts activation memory. But there's a tradeoff: larger batch sizes generally lead to more stable training.

To balance memory constraints with training quality, you can use gradient accumulation—a technique that simulates a larger batch size by accumulating gradients over multiple smaller batches. For example, all of the following configurations result in the same effective batch size (16):

```python
from trl import SFTConfig

training_args = SFTConfig(per_device_train_batch_size=16, gradient_accumulation_steps=1, ...)  # fast but memory intensive
training_args = SFTConfig(per_device_train_batch_size=8, gradient_accumulation_steps=2, ...)
training_args = SFTConfig(per_device_train_batch_size=4, gradient_accumulation_steps=4, ...)
training_args = SFTConfig(per_device_train_batch_size=2, gradient_accumulation_steps=8, ...)
training_args = SFTConfig(per_device_train_batch_size=1, gradient_accumulation_steps=16, ...)  # slow but memory efficient
```

Just note: more accumulation steps mean more forward passes per update, which increases training time. When possible, prefer fewer steps with a larger batch size.



### 🕰 Use gradient checkpointing

Gradient checkpointing is a memory-saving technique that reduces GPU usage during training by selectively storing only some intermediate activations and recomputing the others during backpropagation.

![](https://github.com/cybertronai/gradient-checkpointing/raw/master/img/output.gif)

![](https://github.com/cybertronai/gradient-checkpointing/raw/master/img/output2.gif)

We won't go into the details of how it works, but remember that it can be a bit slower than the standard approach. However, it’s a great way to save memory, especially when training large models. And it's super easy to enable in the `SFTTrainer`, you just have to set the `gradient_checkpointing` argument to `True` when creating the config:

```python
from trl import SFTConfig

config = SFTConfig(gradient_checkpointing=True,  ...)
```

### 🧊 Use mixed precision

Mixed precision training speeds up training and reduces memory usage by combining 16-bit (`float16` or `bfloat16`) and 32-bit (`float32`) floating-point arithmetic. It keeps most operations in 16-bit to save memory and compute time, while using 32-bit where higher precision is needed (like loss scaling or certain model updates). This technique is especially helpful when training large models, as it allows you to fit larger batches or longer sequences in memory—often without a noticeable drop in model performance.

```python
from trl import SFTConfig

training_args = SFTConfig(bf16=True, ...)
```

## 🛸 Did rickification work?

Let's see with a basic question!

In [ ]:
from transformers import pipeline

pipeline = pipeline(task="text-generation", model="qgallouedec/SmolLM2-360M-Rickified")

question = "What would happen to time for an astronaut traveling near the speed of light?"
prompt = [{"role": "user", "content": question}]
pipeline(prompt, max_new_tokens=400)[0]["generated_text"]

It sounds very much like Rick!
Let's try with another one, a bit out-of-distribution, to see how the model reacts.

In [ ]:
question = "What is Edmonton's average temperature in January?"
prompt = [{"role": "user", "content": question}]
pipeline(prompt, max_new_tokens=400)[0]["generated_text"]

Before we close this chapter, let's generate a final one, just for fun!

In [ ]:
question = "A ball is thrown vertically upward with an initial speed of 12 m/s. What is its maximum height?"
prompt = [{"role": "user", "content": question}]
pipeline(prompt, max_new_tokens=400)[0]["generated_text"]